# Create the AKS cluster

In this notebook we'll setup the AKS cluster. To do so, we'll do the following:
1. check that there is enough quota to provision our desired cluster
2. provision the cluster using the `az cli`
3. set up blob fuse on the nodes so the pods in our kubernetes cluster can access our blob storage container

---

### Import packages and load .env

In [1]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import subprocess
import json
import os

In [2]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

True

### Provision AKS cluster and set up blobfuse

Set how many nodes you want to provision.

In [3]:
node_count = 5
set_key(env_path, "NODE_COUNT", str(node_count))

(True, 'NODE_COUNT', '5')

Check that there are enough core of the "Standard_NC6s_v3". If not, check that there are enough core of the "Standard_D2s_v3". If not, raise exception. 

In [4]:
vm_dict = {
    "NCSv3": {
        "size": "Standard_NC6s_v3",
        "cores": 6
    },
    "DSv3": {
        "size": "Standard_D2s_v3",
        "cores": 2
    }
}

print("Checking quota for family size NCSv3...")
vm_family = "NCSv3"
requested_cores = node_count * vm_dict[vm_family]["cores"]

def check_quota(vm_family):
    """
    returns quota object
    """
    results = subprocess.run([
        "az", "vm", "list-usage", 
        "--location", get_key(env_path, "REGION"), 
        "--query", "[?contains(localName, '%s')].{max:limit, current:currentValue}" % (vm_family)
    ], stdout=subprocess.PIPE)
    quota = json.loads(''.join(results.stdout.decode('utf-8')))
    return int(quota[0]['max']) - int(quota[0]['current'])

diff = check_quota(vm_family)
if diff <= requested_cores:
    print("Not enough cores of NCSv3 in region, asking for {} but have {}".format(requested_cores, diff))
    
    print("Retrying with family size DSv3...")
    vm_family = "DSv3"
    requested_cores = node_count * vm_dict[vm_family]["cores"]
    
    diff = check_quota(vm_family)
    if diff <= requested_cores:
        print("Not enough cores of DSv3 in region, asking for {} but have {}".format(requested_cores, diff))
        raise Exception("Core Limit", "Note enough cores to satisfy request")

print("There are enough cores, you may continue...") 

Checking quota for family size NCSv3...
There are enough cores, you may continue...


Create the aks cluster. This step may take a while... Please note that this step creates another resource group in your subscription containing the actual compute of the AKS cluster.

In [6]:
!az aks create \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --name {get_key(env_path, "AKS_CLUSTER")} \
    --node-count {node_count} \
    --node-vm-size {vm_dict[vm_family]["size"]} \
    --generate-ssh-keys \
    --service-principal {get_key(env_path, "SP_CLIENT")} \
    --client-secret {get_key(env_path, "SP_SECRET")}

Install Kubectl - this tool is used to manage the kubernetes cluster.

In [7]:
!sudo az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


In [8]:
!az aks get-credentials \
    --resource-group {get_key(env_path, 'RESOURCE_GROUP')}\
    --name {get_key(env_path, 'AKS_CLUSTER')}

Merged "batchscoringdlcluster" as current context in /home/jiata/.kube/config


Check also that the nodes are up and ready using this command. You may choose to run this command in a new cell.
```bash
!kubectl get nodes
```

### Blobfuse on AKS

Now we setup our AKS cluster so that we have blob storage mounted onto the nodes using blob fuse. More info [here](https://github.com/Azure/kubernetes-volume-drivers/tree/master/flexvolume/blobfuse).

Install blobfuse driver on every agent VM.

In [9]:
!kubectl create -f https://raw.githubusercontent.com/Azure/kubernetes-volume-drivers/master/flexvolume/blobfuse/deployment/blobfuse-flexvol-installer-1.9.yaml

namespace/flex created
daemonset.apps/blobfuse-flexvol-installer created


Check daemonset status.

In [10]:
!kubectl describe daemonset blobfuse-flexvol-installer --namespace=flex
!kubectl get po --namespace=flex -o wide

Name:           blobfuse-flexvol-installer
Selector:       name=blobfuse
Node-Selector:  beta.kubernetes.io/os=linux
Labels:         k8s-app=blobfuse
Annotations:    deprecated.daemonset.template.generation: 1
Desired Number of Nodes Scheduled: 5
Current Number of Nodes Scheduled: 5
Number of Nodes Scheduled with Up-to-date Pods: 5
Number of Nodes Scheduled with Available Pods: 5
Number of Nodes Misscheduled: 0
Pods Status:  5 Running / 0 Waiting / 0 Succeeded / 0 Failed
Pod Template:
  Labels:  name=blobfuse
  Containers:
   blobfuse-flexvol-installer:
    Image:        mcr.microsoft.com/k8s/flexvolume/blobfuse-flexvolume
    Port:         <none>
    Host Port:    <none>
    Environment:  <none>
    Mounts:
      /etc/kubernetes/volumeplugins/ from volplugins (rw)
      /var/log/ from varlog (rw)
  Volumes:
   varlog:
    Type:          HostPath (bare host directory volume)
    Path:          /var/log/
    HostPathType:  
   volplugins:
    Type:          HostPath (bare host directory

Set up credentials for blobfuse.

In [11]:
!kubectl create secret generic blobfusecreds \
    --from-literal accountname={get_key(env_path, 'STORAGE_ACCOUNT_NAME')} \
    --from-literal accountkey={get_key(env_path, 'STORAGE_ACCOUNT_KEY')} \
    --type="azure/blobfuse"

secret/blobfusecreds created


Set the mount directory on our AKS cluster as en dotenv variable.

In [12]:
set_key(env_path, "MOUNT_DIR", "/data")

(True, 'MOUNT_DIR', '/data')

---

Continue to the next [notebook](/notebooks/04_style_transfer_on_aks.ipynb).